# Assumptions (and Notes)

1. Temperature data was not in half-hourly intervals, so we applied the pandas .resample() method.
2. The .resample() method will add any missing half-hourly intervals not initially present in the data
3. If a day (or day that led into another day or days) that had:
  * less then 10 consecutive missing values, we applied the linear interpolator to impute those missing values
    * Days that had 9 or less consecutive missing values didn't lose much information when we applied the linear interpolator. The reason being is that the majority of days that did have missing values only really had 1-3 consecutive half-hourly intervals missing, and they weren't during the peak period, more so in the morning/evening.
  * greater than or equal to 10 consecutive missing values we just removed that day (or those days).
    * Days that had in excess of 10 consecutive missing values, we found hard to apply any interpolation without causing any bias. For example, if times from 10:30am to 6:30pm were missing, and we applied the linear interpolator, then we would miss valuable information during the middle of the day when the temperature usually reaches its peak. Thats why we just removed the entire day.

# Loading Libraries and Data

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from tensorflow import keras
from numpy import array

In [2]:
#######################################################
############### If using Google Drive #################
#######################################################

# Mount Google Drive where datasets are located
from google.colab import drive
drive.mount('/content/gdrive')

# Please note that if this cell does not run go to where the shared folder is on Google Drive, and
# right-click on the shared folder, and select Add shortcut to Drive. Then try execute the cell again.

# Change the current working directory
%cd /content/gdrive/MyDrive/DS\ Capstone\ Project/REPORT_CODE_STRUCTURED/data/

# Define data path
data_path = "processed/"

Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/107U69c8Nh3fH_vc0lG1KAZi0i92sHvcb/DS Capstone Project/REPORT_CODE_STRUCTURED/data


In [ ]:
#######################################################
############### If using Local Computer ###############
#######################################################

# Define data path
data_path = "../data/processed/"

In [3]:
# Import the data
df = pd.read_csv(f'{data_path}data_used_to_build_model.csv')

In [4]:
# Filter just for region of QLD
df_qld = df[df['REGIONID'] == 'QLD'].reset_index(drop=True).copy()

In [5]:
# Check data info
df_qld.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195697 entries, 0 to 195696
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   DATETIME     195697 non-null  object 
 1   TEMPERATURE  195697 non-null  float64
 2   REGIONID     195697 non-null  object 
 3   TOTALDEMAND  195697 non-null  float64
dtypes: float64(2), object(2)
memory usage: 6.0+ MB


In [ ]:
# Print first and last 2 rows of DataFrame
head_tail = [df_qld.head(2), df_qld.tail(2)]
for i in head_tail:
  print(f'{i}\n')

              DATETIME  TEMPERATURE REGIONID  TOTALDEMAND
0  2010-01-01 00:00:00         23.6      QLD      5561.21
1  2010-01-01 00:30:00         23.7      QLD      5422.25

                   DATETIME  TEMPERATURE REGIONID  TOTALDEMAND
195695  2021-03-17 23:30:00         19.6      QLD      5897.64
195696  2021-03-18 00:00:00         19.5      QLD      5737.03



In [ ]:
# Change DATETIME to data type datetime64[ns]
df_qld.loc[:, 'DATETIME'] = pd.to_datetime(df_qld.loc[:, 'DATETIME'], format='%Y-%m-%d %H:%M:%S')

# Feature Selection/Engineering

In [ ]:
### Create 2 copies of the DataFrame

# First copy used for machine learning
dfm = df_qld.copy()

# Second copy used to add back predictions from model at end
dfm2 = df_qld.copy()

In [ ]:
# Create new features
def create_new_features(df):
    df["HOUR"] = df["DATETIME"].dt.hour
    df["ISO_DAYOFWEEK"] = df["DATETIME"].dt.isocalendar().day
    df["MONTH"] = df["DATETIME"].dt.month
    df["QUARTER"] = df["DATETIME"].dt.quarter
    df["DAYOFYEAR"] = df["DATETIME"].dt.dayofyear
    df["DAYOFMONTH"] = df["DATETIME"].dt.day
    df["ISO_WEEKOFYEAR"] = df["DATETIME"].dt.isocalendar().week
    df["SEASON"] = df["DATETIME"].dt.month%12 // 3 + 1 # Season 1 is Summer
    return df

dfm = create_new_features(dfm)

In [ ]:
# Let's check the new column Dtypes
dfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195697 entries, 0 to 195696
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   DATETIME        195697 non-null  datetime64[ns]
 1   TEMPERATURE     195697 non-null  float64       
 2   REGIONID        195697 non-null  object        
 3   TOTALDEMAND     195697 non-null  float64       
 4   HOUR            195697 non-null  int64         
 5   ISO_DAYOFWEEK   195697 non-null  UInt32        
 6   MONTH           195697 non-null  int64         
 7   QUARTER         195697 non-null  int64         
 8   DAYOFYEAR       195697 non-null  int64         
 9   DAYOFMONTH      195697 non-null  int64         
 10  ISO_WEEKOFYEAR  195697 non-null  UInt32        
 11  SEASON          195697 non-null  int64         
dtypes: UInt32(2), datetime64[ns](1), float64(2), int64(6), object(1)
memory usage: 16.8+ MB


In [ ]:
# Check to make sure that:
# (1) there are no values below 0, otherwise MinMaxScaler won't work
# (2) the data types are correct
# (3) we can change the data type to a lower bit number for performance 
dfm.describe()

,TEMPERATURE,TOTALDEMAND,HOUR,ISO_DAYOFWEEK,MONTH,QUARTER,DAYOFYEAR,DAYOFMONTH,ISO_WEEKOFYEAR,SEASON
count,195697.000000,195697.000000,195697.000000,195697.000000,195697.000000,195697.000000,195697.000000,195697.000000,195697.000000,195697.000000
mean,20.413124,6026.646686,11.499941,4.002453,6.435106,2.480493,180.436558,15.687282,26.238333,2.478536
std,5.635223,868.846227,6.922235,1.999140,3.480079,1.126503,106.388310,8.786911,15.210316,1.120130
min,1.300000,3748.240000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,16.600000,5368.680000,5.000000,2.000000,3.000000,1.000000,87.000000,8.000000,13.000000,1.000000
50%,20.900000,5993.390000,11.000000,4.000000,6.000000,2.000000,179.000000,16.000000,26.000000,2.000000
75%,24.400000,6601.630000,17.000000,6.000000,9.000000,3.000000,273.000000,23.000000,39.000000,3.000000
max,42.400000,9988.090000,23.000000,7.000000,12.000000,4.000000,366.000000,31.000000,53.000000,4.000000


In [ ]:
# Remove unnecessary columns - only contains the value QLD
dfm.drop(columns=['REGIONID'], axis=1, inplace=True)

# Transforming feature data types to make DataFrame smaller in size for performance
### int8 - handle values between -128 to 127
### int16 - handle values between -32768 to 32767 
dfm = dfm.astype({'HOUR': 'int8',
                 'ISO_DAYOFWEEK': 'int8',
                 'MONTH': 'int8',
                 'QUARTER': 'int8',
                 'DAYOFYEAR': 'int16',
                 'DAYOFMONTH': 'int8',
                 'ISO_WEEKOFYEAR': 'int8',
                 'SEASON': 'int8'})

# Changing positioning of columns (needed as split_sequences function depends on positioning)
# These four lines of code will make DATETIME and TOTALDEMAND the first and second feature respectively
col = dfm.pop("TOTALDEMAND")
dff = dfm.insert(0, col.name, col)
col = dfm.pop("DATETIME")
dff = dfm.insert(0, col.name, col)

In [ ]:
# Checking the data
dfm.head(2)

,DATETIME,TOTALDEMAND,TEMPERATURE,HOUR,ISO_DAYOFWEEK,MONTH,QUARTER,DAYOFYEAR,DAYOFMONTH,ISO_WEEKOFYEAR,SEASON
0,2010-01-01 00:00:00,5561.21,23.6,0,5,1,1,1,1,53,1
1,2010-01-01 00:30:00,5422.25,23.7,0,5,1,1,1,1,53,1


In [ ]:
# Creates 6 new features, a 30, 60, 90, 120, 150, and 180 lagged version of TOTALDEMAND feature - these 6 will be our response
for i in range(1, 7):
  dfm[f'plus_{i*30}'] = dfm['TOTALDEMAND'].shift(-1*i)

# Drop the rows with missing values
dfm.dropna(inplace=True)

In [ ]:
# Observe the new data
dfm

,DATETIME,TOTALDEMAND,TEMPERATURE,HOUR,ISO_DAYOFWEEK,MONTH,QUARTER,DAYOFYEAR,DAYOFMONTH,ISO_WEEKOFYEAR,SEASON,plus_30,plus_60,plus_90,plus_120,plus_150,plus_180
0,2010-01-01 00:00:00,5561.21,23.6,0,5,1,1,1,1,53,1,5422.25,5315.98,5186.70,5050.83,4924.74,4833.84
1,2010-01-01 00:30:00,5422.25,23.7,0,5,1,1,1,1,53,1,5315.98,5186.70,5050.83,4924.74,4833.84,4815.04
2,2010-01-01 01:00:00,5315.98,23.5,1,5,1,1,1,1,53,1,5186.70,5050.83,4924.74,4833.84,4815.04,4816.91
3,2010-01-01 01:30:00,5186.70,22.6,1,5,1,1,1,1,53,1,5050.83,4924.74,4833.84,4815.04,4816.91,4791.08
4,2010-01-01 02:00:00,5050.83,22.0,2,5,1,1,1,1,53,1,4924.74,4833.84,4815.04,4816.91,4791.08,4772.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195686,2021-03-17 19:00:00,7486.67,20.1,19,3,3,1,76,17,11,2,7327.30,7190.86,7086.78,6894.85,6651.18,6443.62
195687,2021-03-17 19:30:00,7327.30,19.8,19,3,3,1,76,17,11,2,7190.86,7086.78,6894.85,6651.18,6443.62,6264.63
195688,2021-03-17 20:00:00,7190.86,19.8,20,3,3,1,76,17,11,2,7086.78,6894.85,6651.18,6443.62,6264.63,6144.16
195689,2021-03-17 20:30:00,7086.78,19.8,20,3,3,1,76,17,11,2,6894.85,6651.18,6443.62,6264.63,6144.16,5897.64


In [ ]:
# Define model start DATETIME - we only use data from 2016 onwards to build our model
model_start_time = '2016-01-01 00:00:00'

# Define the train/test cutoff DATETIME - used just for final model once we find optimal hyperparameters 
trainTestSplit = '2020-01-01 00:00:00'

# Discard all rows with DATETIME before 'model_start_time' - earlier dates are not an accurate representation of later dates
dfm = dfm[dfm['DATETIME'] >= model_start_time].reset_index(drop=True)

# We also make a further 3 DataFrames for 3-Fold Nested CV
### First we define the endtimes for train/val/test splits
train_end_1618, val_end_1618, test_end_1618 = '2016-12-31 23:30:00', '2017-12-31 23:30:00', '2018-12-31 23:30:00'
train_end_1619, val_end_1619, test_end_1619 = '2017-12-31 23:30:00', '2018-12-31 23:30:00', '2019-12-31 23:30:00'
train_end_1620, val_end_1620, test_end_1620 = '2018-12-31 23:30:00', '2019-12-31 23:30:00', '2021-03-18 00:00:00'

### Define the 3 CV DataFrames
df_1618 = dfm.loc[(model_start_time <= dfm['DATETIME']) & (dfm['DATETIME'] <= test_end_1618), :].copy()
df_1619 = dfm.loc[(model_start_time <= dfm['DATETIME']) & (dfm['DATETIME'] <= test_end_1619), :].copy()
df_1620 = dfm.loc[(model_start_time <= dfm['DATETIME']) & (dfm['DATETIME'] <= test_end_1620), :].copy()

# Create a copy
dfm3 = dfm.copy()
dfm3.index = dfm3['DATETIME']

In [ ]:
# Columns to scale (where adding the six lagged TOTALDEMAND features)
cols = ['TOTALDEMAND', 'TEMPERATURE', 'HOUR', 'ISO_DAYOFWEEK', 'MONTH', 'QUARTER', 'DAYOFYEAR', 'DAYOFMONTH', 'ISO_WEEKOFYEAR', 'SEASON'] + list(dfm.columns[11:])

# Scale main DataFrame - used for final model once we find optimal hyperparameters
scaler = MinMaxScaler()
dfm.loc[dfm['DATETIME'] < trainTestSplit, cols] = scaler.fit_transform(dfm.loc[dfm['DATETIME'] < trainTestSplit, cols])
dfm.loc[dfm['DATETIME'] >= trainTestSplit, cols] = scaler.transform(dfm.loc[dfm['DATETIME'] >= trainTestSplit, cols])

# Scale 3 CV DataFrames
# Both val and test sets will use the same scaled parameters learnt from training set
### 1618
scaler_1618 = MinMaxScaler()
df_1618.loc[df_1618['DATETIME'] <= train_end_1618, cols] = scaler_1618.fit_transform(df_1618.loc[df_1618['DATETIME'] <= train_end_1618, cols])
df_1618.loc[df_1618['DATETIME'] > train_end_1618, cols] = scaler_1618.transform(df_1618.loc[df_1618['DATETIME'] > train_end_1618, cols])

### 1619
scaler_1619 = MinMaxScaler()
df_1619.loc[df_1619['DATETIME'] <= train_end_1619, cols] = scaler_1619.fit_transform(df_1619.loc[df_1619['DATETIME'] <= train_end_1619, cols])
df_1619.loc[df_1619['DATETIME'] > train_end_1619, cols] = scaler_1619.transform(df_1619.loc[df_1619['DATETIME'] > train_end_1619, cols])

### 1620
scaler_1620 = MinMaxScaler()
df_1620.loc[df_1620['DATETIME'] <= train_end_1620, cols] = scaler_1620.fit_transform(df_1620.loc[df_1620['DATETIME'] <= train_end_1620, cols])
df_1620.loc[df_1620['DATETIME'] > train_end_1620, cols] = scaler_1620.transform(df_1620.loc[df_1620['DATETIME'] > train_end_1620, cols])

In [ ]:
# Create function to scale predictions back to original scale - only used for final model once we find optimal hyperparameters
scale_min = scaler.data_min_[10:]
scale_max = scaler.data_max_[10:]

def inv_transform(x, min_=scale_min, max_=scale_max):
    return x * (max_ - min_) + min_

In [ ]:
# Check to make sure each DataFrame did scale 
dfm
#df_1618
#df_1619
#df_1620

,DATETIME,TOTALDEMAND,TEMPERATURE,HOUR,ISO_DAYOFWEEK,MONTH,QUARTER,DAYOFYEAR,DAYOFMONTH,ISO_WEEKOFYEAR,SEASON,plus_30,plus_60,plus_90,plus_120,plus_150,plus_180
0,2016-01-01 00:00:00,0.267854,0.457801,0.000000,0.666667,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.239019,0.222507,0.193202,0.183580,0.173685,0.164083
1,2016-01-01 00:30:00,0.239019,0.468031,0.000000,0.666667,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.222507,0.193202,0.183580,0.173685,0.164083,0.157930
2,2016-01-01 01:00:00,0.222507,0.455243,0.043478,0.666667,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.193202,0.183580,0.173685,0.164083,0.157930,0.150700
3,2016-01-01 01:30:00,0.193202,0.434783,0.043478,0.666667,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.183580,0.173685,0.164083,0.157930,0.150700,0.152100
4,2016-01-01 02:00:00,0.183580,0.429668,0.086957,0.666667,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.173685,0.164083,0.157930,0.150700,0.152100,0.151235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90758,2021-03-17 19:00:00,0.569813,0.475703,0.826087,0.333333,0.181818,0.0,0.205479,0.533333,0.192308,0.333333,0.542405,0.518941,0.501041,0.468034,0.426128,0.390433
90759,2021-03-17 19:30:00,0.542405,0.468031,0.826087,0.333333,0.181818,0.0,0.205479,0.533333,0.192308,0.333333,0.518941,0.501041,0.468034,0.426128,0.390433,0.359650
90760,2021-03-17 20:00:00,0.518941,0.468031,0.869565,0.333333,0.181818,0.0,0.205479,0.533333,0.192308,0.333333,0.501041,0.468034,0.426128,0.390433,0.359650,0.338932
90761,2021-03-17 20:30:00,0.501041,0.468031,0.869565,0.333333,0.181818,0.0,0.205479,0.533333,0.192308,0.333333,0.468034,0.426128,0.390433,0.359650,0.338932,0.296537


In [ ]:
# Specifies the number of timesteps (i.e. half-hour intervals) to batch data by
n_steps_in = 48

# Split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, trainTestSplit, train_end=None, val_end=None, test_end=None, trainValTest=False):
    
    if trainValTest is False:

        X_train, y_train = list(), list()
        X_test, y_test = list(), list()
        for i in range(len(sequences)):

            # find the end of this pattern
            end_ix = i + n_steps_in

            # check if we are beyond the dataset
            if end_ix > len(sequences):
                break

            # Gets the DATETIME of the prediction in which the model will make 
            pred_datetime = str(sequences['DATETIME'][end_ix-1])
            
            # Defines training data
            if pred_datetime < trainTestSplit:
                # gather input and output parts of the pattern
                seq_x, seq_y = sequences.iloc[i:end_ix, 1:11], sequences.iloc[end_ix-1, 11:]
                X_train.append(seq_x)
                y_train.append(seq_y)

            # Define testing data
            elif pred_datetime >= trainTestSplit:
                # gather input and output parts of the pattern
                seq_x, seq_y = sequences.iloc[i:end_ix, 1:11], sequences.iloc[end_ix-1, 11:]
                X_test.append(seq_x)
                y_test.append(seq_y)

        return array(X_train), array(y_train), array(X_test), array(y_test)

    else:

        X_train, y_train = list(), list()
        X_val, y_val = list(), list()
        X_test, y_test = list(), list()
        for i in range(len(sequences)):

            # find the end of this pattern
            end_ix = i + n_steps_in

            # check if we are beyond the dataset
            if end_ix > len(sequences):
                break

            # Gets the DATETIME of the prediction in which the model will make 
            pred_datetime = str(sequences['DATETIME'][end_ix-1])
            
            # Defines training data
            if pred_datetime <= train_end:
                # gather input and output parts of the pattern
                seq_x, seq_y = sequences.iloc[i:end_ix, 1:11], sequences.iloc[end_ix-1, 11:]
                X_train.append(seq_x)
                y_train.append(seq_y)

            # Define testing data
            elif (train_end < pred_datetime) and (pred_datetime <= val_end):
                # gather input and output parts of the pattern
                seq_x, seq_y = sequences.iloc[i:end_ix, 1:11], sequences.iloc[end_ix-1, 11:]
                X_val.append(seq_x)
                y_val.append(seq_y)

            elif (val_end < pred_datetime) and (pred_datetime <= test_end):
                # gather input and output parts of the pattern
                seq_x, seq_y = sequences.iloc[i:end_ix, 1:11], sequences.iloc[end_ix-1, 11:]
                X_test.append(seq_x)
                y_test.append(seq_y)
    
        return array(X_train), array(y_train), array(X_val), array(y_val), array(X_test), array(y_test)

In [ ]:
# This cell will take a few minutes to run

# Create main train/test dataframes
X_train, y_train, X_test, y_test = split_sequences(dfm, n_steps_in, trainTestSplit) 

# Create train/validation dataframes
X_train_1618, y_train_1618, X_val_1618, y_val_1618, X_test_1618, y_test_1618 = split_sequences(df_1618, n_steps_in, trainTestSplit=None,
                                                                                               train_end=train_end_1618, val_end=val_end_1618, test_end=test_end_1618, trainValTest=True)
X_train_1619, y_train_1619, X_val_1619, y_val_1619, X_test_1619, y_test_1619 = split_sequences(df_1619, n_steps_in, trainTestSplit=None,
                                                                                               train_end=train_end_1619, val_end=val_end_1619, test_end=test_end_1619, trainValTest=True) 
X_train_1620, y_train_1620, X_val_1620, y_val_1620, X_test_1620, y_test_1620 = split_sequences(df_1620, n_steps_in, trainTestSplit=None,
                                                                                               train_end=train_end_1620, val_end=val_end_1620, test_end=test_end_1620, trainValTest=True)

In [ ]:
print('16-21')
print(f'Train:\nX has shape: {X_train.shape}\ny has shape: {y_train.shape}\n')
print(f'Test:\nX has shape: {X_test.shape}\ny has shape: {y_test.shape}')

print('\n\n16-18')
print(f'Train:\nX has shape: {X_train_1618.shape}\ny has shape: {y_train_1618.shape}\n')
print(f'Validation:\nX has shape: {X_val_1618.shape}\ny has shape: {y_val_1618.shape}')
print(f'Test:\nX has shape: {X_test_1618.shape}\ny has shape: {y_test_1618.shape}')

print('\n\n16-19')
print(f'Train:\nX has shape: {X_train_1619.shape}\ny has shape: {y_train_1619.shape}\n')
print(f'Validation:\nX has shape: {X_val_1619.shape}\ny has shape: {y_val_1619.shape}')
print(f'Test:\nX has shape: {X_test_1619.shape}\ny has shape: {y_test_1619.shape}')

print('\n\n16-20')
print(f'Train:\nX has shape: {X_train_1620.shape}\ny has shape: {y_train_1620.shape}\n')
print(f'Validation:\nX has shape: {X_val_1620.shape}\ny has shape: {y_val_1620.shape}')
print(f'Test:\nX has shape: {X_test_1620.shape}\ny has shape: {y_test_1620.shape}')

16-21
Train:
X has shape: (69553, 48, 10)
y has shape: (69553, 6)

Test:
X has shape: (21163, 48, 10)
y has shape: (21163, 6)


16-18
Train:
X has shape: (16993, 48, 10)
y has shape: (16993, 6)

Validation:
X has shape: (17520, 48, 10)
y has shape: (17520, 6)
Test:
X has shape: (17520, 48, 10)
y has shape: (17520, 6)


16-19
Train:
X has shape: (34513, 48, 10)
y has shape: (34513, 6)

Validation:
X has shape: (17520, 48, 10)
y has shape: (17520, 6)
Test:
X has shape: (17520, 48, 10)
y has shape: (17520, 6)


16-20
Train:
X has shape: (52033, 48, 10)
y has shape: (52033, 6)

Validation:
X has shape: (17520, 48, 10)
y has shape: (17520, 6)
Test:
X has shape: (21163, 48, 10)
y has shape: (21163, 6)


In [ ]:
# Convert each of the numpy arrays to tensors
X_train_tf = tf.convert_to_tensor(X_train, np.float32)
y_train_tf = tf.convert_to_tensor(y_train, np.float32)
X_test_tf = tf.convert_to_tensor(X_test, np.float32) 
y_test_tf = tf.convert_to_tensor(y_test, np.float32) 

X_train_tf_1618 = tf.convert_to_tensor(X_train_1618, np.float32)
y_train_tf_1618 = tf.convert_to_tensor(y_train_1618, np.float32)
X_val_tf_1618 = tf.convert_to_tensor(X_val_1618, np.float32) 
y_val_tf_1618 = tf.convert_to_tensor(y_val_1618, np.float32) 
X_test_tf_1618 = tf.convert_to_tensor(X_test_1618, np.float32) 
y_test_tf_1618 = tf.convert_to_tensor(y_test_1618, np.float32) 

X_train_tf_1619 = tf.convert_to_tensor(X_train_1619, np.float32)
y_train_tf_1619 = tf.convert_to_tensor(y_train_1619, np.float32)
X_val_tf_1619 = tf.convert_to_tensor(X_val_1619, np.float32) 
y_val_tf_1619 = tf.convert_to_tensor(y_val_1619, np.float32) 
X_test_tf_1619 = tf.convert_to_tensor(X_test_1619, np.float32) 
y_test_tf_1619 = tf.convert_to_tensor(y_test_1619, np.float32)

X_train_tf_1620 = tf.convert_to_tensor(X_train_1620, np.float32)
y_train_tf_1620 = tf.convert_to_tensor(y_train_1620, np.float32)
X_val_tf_1620 = tf.convert_to_tensor(X_val_1620, np.float32) 
y_val_tf_1620 = tf.convert_to_tensor(y_val_1620, np.float32) 
X_test_tf_1620 = tf.convert_to_tensor(X_test_1620, np.float32) 
y_test_tf_1620 = tf.convert_to_tensor(y_test_1620, np.float32)

In [ ]:
# Create train and test Datasets batched with 144 observations (48 half-hours/day * 3 days)
batch_size = 48*3  

dataset_train_tf = tf.data.Dataset.from_tensor_slices((X_train_tf, y_train_tf)).batch(batch_size)
dataset_test_tf = tf.data.Dataset.from_tensor_slices((X_test_tf, y_test_tf)).batch(batch_size)

dataset_train_tf_1618 = tf.data.Dataset.from_tensor_slices((X_train_tf_1618, y_train_tf_1618)).batch(batch_size)
dataset_val_tf_1618 = tf.data.Dataset.from_tensor_slices((X_val_tf_1618, y_val_tf_1618)).batch(batch_size)
dataset_test_tf_1618 = tf.data.Dataset.from_tensor_slices((X_test_tf_1618, y_test_tf_1618)).batch(batch_size)

dataset_train_tf_1619 = tf.data.Dataset.from_tensor_slices((X_train_tf_1619, y_train_tf_1619)).batch(batch_size)
dataset_val_tf_1619 = tf.data.Dataset.from_tensor_slices((X_val_tf_1619, y_val_tf_1619)).batch(batch_size)
dataset_test_tf_1619 = tf.data.Dataset.from_tensor_slices((X_test_tf_1619, y_test_tf_1619)).batch(batch_size)

dataset_train_tf_1620 = tf.data.Dataset.from_tensor_slices((X_train_tf_1620, y_train_tf_1620)).batch(batch_size)
dataset_val_tf_1620 = tf.data.Dataset.from_tensor_slices((X_val_tf_1620, y_val_tf_1620)).batch(batch_size)
dataset_test_tf_1620 = tf.data.Dataset.from_tensor_slices((X_test_tf_1620, y_test_tf_1620)).batch(batch_size)

## Defining the 3-Fold Nested CV Loop

In [ ]:
# Define callback for early stopping
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                            patience=3,
                                            restore_best_weights=True,
                                            min_delta=0.005)

In [ ]:
# Define model inside function
def run_model_trainVal(df_train, df_val, df_test, epochs, n_features, activation, hid_neurons, optimizer, n_steps_in=n_steps_in, n_steps_out=6):
    # Define dict to put in train, val, and test accuracies/losses
    trainValTestLoss = dict()
    
    # Define model
    model = Sequential()
    model.add(LSTM(hid_neurons, activation=activation, return_sequences=True, input_shape=(n_steps_in, n_features)))
    model.add(LSTM(hid_neurons, activation=activation))
    model.add(Dense(n_steps_out))

    # Compile model
    # model.compile(optimizer=optimizer, loss=root_mean_squared_error)
    model.compile(optimizer=optimizer, loss='mean_absolute_error')

    # Fit model
    history = model.fit(df_train, validation_data=df_val, epochs=epochs, callbacks=[callback])

    # Evaluate model on testing data
    test_loss = model.evaluate(df_test)

    # Update trainValTestAcc dict
    trainValTestLoss.update(history.history)
    trainValTestLoss['test_loss'] = test_loss

    return trainValTestLoss

In [ ]:
#################################################################################
########### You will need to manually change the parameters each time ###########
#################################################################################
epochs = 50
n_features = X_train.shape[2]
activation = 'tanh' # GPU only works for tanh activation
hid_neurons = 50
optimizer = keras.optimizers.Adam(learning_rate=0.001)
# Don't forget "batch_size" three code cells above!

datasets = {
    
    'model_1618': {
        'df_train': dataset_train_tf_1618,
        'df_val': dataset_val_tf_1618,
        'df_test': dataset_test_tf_1618,
    },
    
    'model_1619': {
        'df_train': dataset_train_tf_1619,
        'df_val': dataset_val_tf_1619,
        'df_test': dataset_test_tf_1619,
    },
    
    'model_1620': {
        'df_train': dataset_train_tf_1620,
        'df_val': dataset_val_tf_1620,
        'df_test': dataset_test_tf_1620,
    }
    
}

model_losses = {}

In [ ]:
for model, dataset in datasets.items():
    print(f'Running {model}\n\n')
    history = run_model_trainVal(df_train = dataset['df_train'], 
                                 df_val = dataset['df_val'], 
                                 df_test = dataset['df_test'],
                                 epochs = epochs, 
                                 n_features = n_features, 
                                 activation = activation,
                                 hid_neurons = hid_neurons,
                                 optimizer = optimizer)
    model_losses[model] = history
    print('\n\n')

Running model_1618


119/119 [==============================] - 18s 123ms/step - loss: 0.1385 - val_loss: 0.1887


KeyboardInterrupt: ignored

In [ ]:
count = 0
train_loss, val_loss, test_loss = 0, 0, 0

for k,v in model_losses.items():
    count += 1

    # Calculate val_loss, and index corresponding to minimum val_loss to get corresponding train_loss 
    val_index = np.argmin(v['val_loss'])
    val_loss += np.min(v['val_loss'])

    train_loss += v['loss'][val_index]
    test_loss += v['test_loss']
  
print(f'Average Training Loss: {train_loss / count}')
print(f'Average Validation Loss: {val_loss / count}')
print(f'Average Testing Loss: {test_loss / count}')

ZeroDivisionError: ignored

# Training and Testing - Using Optimal Hyperparameters from above

In [ ]:
for i, j in dataset_train_tf:
  print(i.shape, j.shape)
  break

(144, 48, 10) (144, 6)


In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                            patience=3,
                                            restore_best_weights=True,
                                            min_delta=0.005)
epochs = 1 # 10, 15, 25

### Define training/testing model
# Define dict to put in train, val, and test accuracies/losses
trainValTestLoss = dict()

# Define model
model = Sequential()
model.add(LSTM(50, activation='tanh', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))) # 48, 10
model.add(LSTM(50, activation='tanh'))
model.add(Dense(6))

# Compile model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), 
              loss='mean_absolute_error')

# Fit model
history = model.fit(dataset_train_tf, epochs=epochs)

# Evaluate model on testing data
test_loss = model.evaluate(dataset_test_tf)

# Update trainValTestAcc dict
trainValTestLoss.update(history.history)
trainValTestLoss['test_loss'] = test_loss

147/147 [==============================] - 6s 32ms/step - loss: 0.0800


In [ ]:
trainValTestLoss

{'loss': [0.0676279217004776], 'test_loss': 0.08002595603466034}

In [ ]:
# Create function to scale predictions back to original scale
scale_min = scaler.data_min_[10:]
scale_max = scaler.data_max_[10:]

def inv_transform(df, min_=scale_min, max_=scale_max):
  for i in range(len(min_)):
    df[:, i] = df[:, i] * (max_[i] - min_[i]) + min_[i]
  return df

In [ ]:
# Calculates the residuals
def residuals(df, max=6*30+30):
  for i in range(30, max, 30):
    df[f'residuals_{i}'] = df[f'plus_{i}'] - df[f'pred_{i}']
  return df

In [ ]:
# Observing DataFrame, which we defined earlier, to append predictions onto
dfm3

,DATETIME,TOTALDEMAND,TEMPERATURE,HOUR,ISO_DAYOFWEEK,MONTH,QUARTER,DAYOFYEAR,DAYOFMONTH,ISO_WEEKOFYEAR,SEASON,plus_30,plus_60,plus_90,plus_120,plus_150,plus_180
DATETIME,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,2016-01-01 00:00:00,5730.86,19.4,0,5,1,1,1,1,53,1,5563.19,5467.18,5296.78,5240.83,5183.29,5127.46
2016-01-01 00:30:00,2016-01-01 00:30:00,5563.19,19.8,0,5,1,1,1,1,53,1,5467.18,5296.78,5240.83,5183.29,5127.46,5091.68
2016-01-01 01:00:00,2016-01-01 01:00:00,5467.18,19.3,1,5,1,1,1,1,53,1,5296.78,5240.83,5183.29,5127.46,5091.68,5049.64
2016-01-01 01:30:00,2016-01-01 01:30:00,5296.78,18.5,1,5,1,1,1,1,53,1,5240.83,5183.29,5127.46,5091.68,5049.64,5057.78
2016-01-01 02:00:00,2016-01-01 02:00:00,5240.83,18.3,2,5,1,1,1,1,53,1,5183.29,5127.46,5091.68,5049.64,5057.78,5052.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-17 19:00:00,2021-03-17 19:00:00,7486.67,20.1,19,3,3,1,76,17,11,2,7327.30,7190.86,7086.78,6894.85,6651.18,6443.62
2021-03-17 19:30:00,2021-03-17 19:30:00,7327.30,19.8,19,3,3,1,76,17,11,2,7190.86,7086.78,6894.85,6651.18,6443.62,6264.63
2021-03-17 20:00:00,2021-03-17 20:00:00,7190.86,19.8,20,3,3,1,76,17,11,2,7086.78,6894.85,6651.18,6443.62,6264.63,6144.16


In [ ]:
# Calculates train and test predictions
preds_train = model.predict(X_train_tf) 
preds_test = model.predict(X_test_tf) 

# Get cols - used to create new DataFrame with predictions appended
colsPred = ['DATETIME', 'TOTALDEMAND'] + [f'plus_{x}' for x in range(30, 7*30, 30)]

# Create new DataFrame with just DATETIMES relating to train predictions, then scale back predictions to original scale, then append Residual columns
train_preds = dfm3[colsPred][n_steps_in-1:len(preds_train)+n_steps_in-1].copy()
train_preds[[f'pred_{x}' for x in range(30, 7*30, 30)]] = inv_transform(preds_train)
train_preds = residuals(train_preds)

# Create new DataFrame with just DATETIMES relating to test predictions, then scale back predictions to original scale, then append Residual columns
test_preds = dfm3[colsPred][len(preds_train)+n_steps_in-1:].copy()
test_preds[[f'pred_{x}' for x in range(30, 7*30, 30)]] = inv_transform(preds_test)
test_preds = residuals(test_preds)

# Adding Residuals / plus (for MAPE)
for i in range(30, 7*30, 30):
  test_preds[f'resid_over_actual_{i}'] = test_preds[f'residuals_{i}'] / test_preds[f'plus_{i}']

In [ ]:
# Observe DataFrame related to training predictions
train_preds.head(2)

,DATETIME,TOTALDEMAND,plus_30,plus_60,plus_90,plus_120,plus_150,plus_180,pred_30,pred_60,pred_90,pred_120,pred_150,pred_180,residuals_30,residuals_60,residuals_90,residuals_120,residuals_150,residuals_180
DATETIME,,,,,,,,,,,,,,,,,,,,
2016-01-01 23:30:00,2016-01-01 23:30:00,5809.78,5625.79,5418.92,5287.29,5123.09,5019.33,5002.42,6431.719727,6057.678711,6053.183594,6222.263672,5914.086914,5949.947754,-805.929727,-638.758711,-765.893594,-1099.173672,-894.756914,-947.527754
2016-01-02 00:00:00,2016-01-02 00:00:00,5625.79,5418.92,5287.29,5123.09,5019.33,5002.42,4961.19,6278.652832,5922.485840,5902.059082,6098.519531,5820.501953,5894.673340,-859.732832,-635.195840,-778.969082,-1079.189531,-818.081953,-933.483340


In [ ]:
# Observe DataFrame related to testing predictions
test_preds.head(2)

,DATETIME,TOTALDEMAND,plus_30,plus_60,plus_90,plus_120,plus_150,plus_180,pred_30,pred_60,pred_90,pred_120,pred_150,pred_180,residuals_30,residuals_60,residuals_90,residuals_120,residuals_150,residuals_180,resid_over_actual_30,resid_over_actual_60,resid_over_actual_90,resid_over_actual_120,resid_over_actual_150,resid_over_actual_180
DATETIME,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01 00:00:00,2020-01-01 00:00:00,6218.39,6029.36,5954.32,5888.68,5820.63,5750.66,5705.83,5939.029785,5903.852539,5669.841309,5778.642578,5593.921875,5465.144531,90.330215,50.467461,218.838691,41.987422,156.738125,240.685469,0.014982,0.008476,0.037163,0.007214,0.027256,0.042182
2020-01-01 00:30:00,2020-01-01 00:30:00,6029.36,5954.32,5888.68,5820.63,5750.66,5705.83,5662.17,5894.525391,5839.777344,5672.261230,5798.119141,5631.458984,5547.957031,59.794609,48.902656,148.368770,-47.459141,74.371016,114.212969,0.010042,0.008305,0.025490,-0.008253,0.013034,0.020171


In [ ]:
# Create functions to calculate RMSE, MSE, MAE, MAPE, R-Squared
def RMSE(dfPreds, cols=[f'residuals_{x}' for x in range(30, 30*6+30, 30)]):
  RMSE = dfPreds[cols].apply(lambda x: x**2, axis=1).agg('mean').apply(lambda x: np.sqrt(x)).mean()
  return RMSE

def MSE(dfPreds, cols=[f'residuals_{x}' for x in range(30, 30*6+30, 30)]):
  MSE = dfPreds[cols].apply(lambda x: x**2, axis=1).agg('mean').mean()
  return MSE

def MAE(dfPreds, cols=[f'residuals_{x}' for x in range(30, 30*6+30, 30)]):
  MAE = dfPreds[cols].apply(lambda x: np.abs(x), axis=1).agg('mean').mean()
  return MAE

def MAPE(dfPreds, cols=[f'resid_over_actual_{x}' for x in range(30, 30*6+30, 30)]):
  MAPE = dfPreds[cols].apply(lambda x: np.abs(x), axis=1).agg('mean').mean()
  return MAPE

def Rsq(df, 
        cols=[f'residuals_{x}' for x in range(30, 30*6+30, 30)],
        cols2=[f'plus_{x}' for x in range(30, 30*6+30, 30)]):
  MSE = df[cols].apply(lambda x: x**2, axis=1).agg('mean').mean()
  VAR = df[cols2].apply(lambda x: np.var(x)).mean()
  R_Squared = 1 - (MSE / VAR)
  return R_Squared

RMSE = RMSE(test_preds)
MSE = MSE(test_preds)
MAE = MAE(test_preds)
MAPE = MAPE(test_preds)
R_Squared = Rsq(test_preds)

In [ ]:
print(f'RMSE: {RMSE}\n'
      f'MSE: {MSE}\n'
      f'MAE: {MAE}\n'
      f'MAPE: {MAPE}\n'
      f'Accuracy: {1 - MAPE}\n'
      f'R Squared: {R_Squared}')

RMSE: 588.341835031303
MSE: 349173.53394970484
MAE: 465.3292480931747
MAPE: 0.07847595483697338
Accuracy: 0.9215240451630267
R Squared: 0.6170770000303334
